### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
from keras.datasets import cifar10
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
num_filter = 32
compression = 0.5

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
# Dense Block
def denseblock(input, num_filter = 32):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
    return temp

## transition Blosck
def transition(input, num_filter = 32):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg



In [0]:
num_filter = 32
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)

BatchNorm = layers.BatchNormalization()(Last_Block)
relu = layers.Activation('relu')(BatchNorm)
AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
conv_output=layers.Conv2D(10,(2,2))(AvgPooling)
conv_output1=layers.Conv2D(10,(1,1))(conv_output)
output=layers.Flatten()(conv_output1)
output = layers.Activation('softmax')(output)


In [58]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_743 (Conv2D)             (None, 32, 32, 32)   864         input_15[0][0]                   
__________________________________________________________________________________________________
batch_normalization_728 (BatchN (None, 32, 32, 32)   128         conv2d_743[0][0]                 
__________________________________________________________________________________________________
activation_745 (Activation)     (None, 32, 32, 32)   0           batch_normalization_728[0][0]    
____________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)

In [0]:
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')

In [0]:
sgd=SGD(lr=0.01, momentum = 0.9)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

**Model: epochs:300 Test Accuracy:89%**

In [62]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_30epochs.h5'))

Epoch 1/30
391/390 [==============================] - 86s 219ms/step - loss: 1.8385 - accuracy: 0.3240 - val_loss: 1.7987 - val_accuracy: 0.3746
Epoch 2/30
391/390 [==============================] - 85s 218ms/step - loss: 1.5182 - accuracy: 0.4459 - val_loss: 1.5324 - val_accuracy: 0.4470
Epoch 3/30
391/390 [==============================] - 85s 218ms/step - loss: 1.3841 - accuracy: 0.5002 - val_loss: 1.4850 - val_accuracy: 0.4671
Epoch 4/30
391/390 [==============================] - 85s 218ms/step - loss: 1.2878 - accuracy: 0.5344 - val_loss: 1.5536 - val_accuracy: 0.4764
Epoch 5/30
391/390 [==============================] - 85s 218ms/step - loss: 1.2062 - accuracy: 0.5648 - val_loss: 1.3894 - val_accuracy: 0.5274
Epoch 6/30
391/390 [==============================] - 86s 219ms/step - loss: 1.1471 - accuracy: 0.5881 - val_loss: 1.3093 - val_accuracy: 0.5445
Epoch 7/30
391/390 [==============================] - 86s 220ms/step - loss: 1.0908 - accuracy: 0.6112 - val_loss: 1.2884 - val_ac

In [63]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_60epochs.h5'))


Epoch 1/30
391/390 [==============================] - 85s 218ms/step - loss: 0.6338 - accuracy: 0.7768 - val_loss: 0.7100 - val_accuracy: 0.7604
Epoch 2/30
391/390 [==============================] - 86s 219ms/step - loss: 0.6247 - accuracy: 0.7812 - val_loss: 0.7236 - val_accuracy: 0.7575
Epoch 3/30
391/390 [==============================] - 85s 218ms/step - loss: 0.6162 - accuracy: 0.7827 - val_loss: 0.7967 - val_accuracy: 0.7382
Epoch 4/30
391/390 [==============================] - 85s 218ms/step - loss: 0.6115 - accuracy: 0.7831 - val_loss: 0.6847 - val_accuracy: 0.7668
Epoch 5/30
391/390 [==============================] - 86s 219ms/step - loss: 0.6013 - accuracy: 0.7921 - val_loss: 0.6477 - val_accuracy: 0.7801
Epoch 6/30
391/390 [==============================] - 86s 219ms/step - loss: 0.5903 - accuracy: 0.7932 - val_loss: 0.7652 - val_accuracy: 0.7449
Epoch 7/30
391/390 [==============================] - 86s 219ms/step - loss: 0.5824 - accuracy: 0.7960 - val_loss: 0.5867 - val_ac

In [64]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_90epochs.h5'))

Epoch 1/30
391/390 [==============================] - 85s 217ms/step - loss: 0.4601 - accuracy: 0.8406 - val_loss: 0.5665 - val_accuracy: 0.8139
Epoch 2/30
391/390 [==============================] - 85s 217ms/step - loss: 0.4611 - accuracy: 0.8396 - val_loss: 0.5623 - val_accuracy: 0.8140
Epoch 3/30
391/390 [==============================] - 85s 217ms/step - loss: 0.4509 - accuracy: 0.8425 - val_loss: 0.6243 - val_accuracy: 0.7931
Epoch 4/30
391/390 [==============================] - 85s 218ms/step - loss: 0.4533 - accuracy: 0.8435 - val_loss: 0.6515 - val_accuracy: 0.7912
Epoch 5/30
391/390 [==============================] - 86s 219ms/step - loss: 0.4448 - accuracy: 0.8440 - val_loss: 0.5613 - val_accuracy: 0.8104
Epoch 6/30
391/390 [==============================] - 86s 220ms/step - loss: 0.4396 - accuracy: 0.8459 - val_loss: 0.6198 - val_accuracy: 0.8026
Epoch 7/30
391/390 [==============================] - 85s 218ms/step - loss: 0.4383 - accuracy: 0.8471 - val_loss: 0.6001 - val_ac

In [65]:

model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_120epochs.h5'))

Epoch 1/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3709 - accuracy: 0.8698 - val_loss: 0.4988 - val_accuracy: 0.8364
Epoch 2/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3689 - accuracy: 0.8708 - val_loss: 0.5581 - val_accuracy: 0.8205
Epoch 3/30
391/390 [==============================] - 85s 218ms/step - loss: 0.3643 - accuracy: 0.8724 - val_loss: 0.5163 - val_accuracy: 0.8325
Epoch 4/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3681 - accuracy: 0.8701 - val_loss: 0.5318 - val_accuracy: 0.8264
Epoch 5/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3605 - accuracy: 0.8748 - val_loss: 0.5757 - val_accuracy: 0.8181
Epoch 6/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3561 - accuracy: 0.8774 - val_loss: 0.5582 - val_accuracy: 0.8253
Epoch 7/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3530 - accuracy: 0.8755 - val_loss: 0.4628 - val_ac

In [66]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_150epochs.h5'))


Epoch 1/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3101 - accuracy: 0.8923 - val_loss: 0.4372 - val_accuracy: 0.8559
Epoch 2/30
391/390 [==============================] - 85s 218ms/step - loss: 0.3104 - accuracy: 0.8897 - val_loss: 0.4566 - val_accuracy: 0.8507
Epoch 3/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3067 - accuracy: 0.8931 - val_loss: 0.4852 - val_accuracy: 0.8471
Epoch 4/30
391/390 [==============================] - 86s 219ms/step - loss: 0.3078 - accuracy: 0.8914 - val_loss: 0.5366 - val_accuracy: 0.8327
Epoch 5/30
391/390 [==============================] - 85s 218ms/step - loss: 0.3023 - accuracy: 0.8954 - val_loss: 0.5466 - val_accuracy: 0.8345
Epoch 6/30
391/390 [==============================] - 85s 218ms/step - loss: 0.3022 - accuracy: 0.8939 - val_loss: 0.5084 - val_accuracy: 0.8406
Epoch 7/30
391/390 [==============================] - 85s 217ms/step - loss: 0.3026 - accuracy: 0.8930 - val_loss: 0.4417 - val_ac

In [67]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_150epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_180epochs.h5'))

Epoch 1/30
391/390 [==============================] - 87s 223ms/step - loss: 0.2433 - accuracy: 0.9137 - val_loss: 0.3915 - val_accuracy: 0.8722
Epoch 2/30
391/390 [==============================] - 86s 219ms/step - loss: 0.2301 - accuracy: 0.9202 - val_loss: 0.3770 - val_accuracy: 0.8757
Epoch 3/30
391/390 [==============================] - 85s 218ms/step - loss: 0.2310 - accuracy: 0.9195 - val_loss: 0.3831 - val_accuracy: 0.8751
Epoch 4/30
391/390 [==============================] - 86s 219ms/step - loss: 0.2264 - accuracy: 0.9193 - val_loss: 0.3884 - val_accuracy: 0.8770
Epoch 5/30
391/390 [==============================] - 86s 219ms/step - loss: 0.2292 - accuracy: 0.9196 - val_loss: 0.3894 - val_accuracy: 0.8759
Epoch 6/30
391/390 [==============================] - 86s 219ms/step - loss: 0.2257 - accuracy: 0.9216 - val_loss: 0.3884 - val_accuracy: 0.8767
Epoch 7/30
391/390 [==============================] - 86s 219ms/step - loss: 0.2274 - accuracy: 0.9200 - val_loss: 0.3821 - val_ac

In [69]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_180epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_190epochs.h5'))

Epoch 1/10
391/390 [==============================] - 86s 220ms/step - loss: 0.2100 - accuracy: 0.9256 - val_loss: 0.3873 - val_accuracy: 0.8776
Epoch 2/10
391/390 [==============================] - 86s 219ms/step - loss: 0.2112 - accuracy: 0.9251 - val_loss: 0.3847 - val_accuracy: 0.8796
Epoch 3/10
391/390 [==============================] - 86s 219ms/step - loss: 0.2089 - accuracy: 0.9278 - val_loss: 0.3867 - val_accuracy: 0.8779
Epoch 4/10
391/390 [==============================] - 85s 219ms/step - loss: 0.2105 - accuracy: 0.9249 - val_loss: 0.3844 - val_accuracy: 0.8788
Epoch 5/10
391/390 [==============================] - 85s 219ms/step - loss: 0.2105 - accuracy: 0.9258 - val_loss: 0.3804 - val_accuracy: 0.8801
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2106 - accuracy: 0.9266 - val_loss: 0.3850 - val_accuracy: 0.8788
Epoch 7/10
391/390 [==============================] - 86s 219ms/step - loss: 0.2071 - accuracy: 0.9275 - val_loss: 0.3820 - val_ac

In [7]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_190epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_200epochs.h5'))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
391/390 [==============================] - 90s 231ms/step - loss: 0.2092 - accuracy: 0.9270 - val_loss: 0.3844 - val_accuracy: 0.8793
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2074 - accuracy: 0.9278 - val_loss: 0.3801 - val_accuracy: 0.8807
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2092 - accuracy: 0.9267 - val_loss: 0.3844 - val_accuracy: 0.8792
Epoch 4/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2096 - accuracy: 0.9258 - val_loss: 0.3794 - val_accuracy: 0.8807
Epoch 5/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2078 - accuracy: 0.9270 - val_loss: 0.3835 - val_accuracy: 0.8801
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2056 - accuracy: 0.9285 - val_loss: 0.3802 - val_accuracy: 0.8802
Epoch 7/10
391/390 [==============================] - 

In [10]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_200epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.00001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_210epochs.h5'))

Epoch 1/10
391/390 [==============================] - 86s 219ms/step - loss: 0.2082 - accuracy: 0.9273 - val_loss: 0.3813 - val_accuracy: 0.8790
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2097 - accuracy: 0.9261 - val_loss: 0.3818 - val_accuracy: 0.8789
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2072 - accuracy: 0.9260 - val_loss: 0.3815 - val_accuracy: 0.8785
Epoch 4/10
391/390 [==============================] - 85s 217ms/step - loss: 0.2084 - accuracy: 0.9264 - val_loss: 0.3845 - val_accuracy: 0.8778
Epoch 5/10
391/390 [==============================] - 85s 217ms/step - loss: 0.2066 - accuracy: 0.9269 - val_loss: 0.3808 - val_accuracy: 0.8794
Epoch 6/10
391/390 [==============================] - 85s 217ms/step - loss: 0.2057 - accuracy: 0.9274 - val_loss: 0.3812 - val_accuracy: 0.8794
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2081 - accuracy: 0.9272 - val_loss: 0.3805 - val_ac

In [11]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_210epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.000001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_220epochs.h5'))

Epoch 1/10
391/390 [==============================] - 85s 219ms/step - loss: 0.2068 - accuracy: 0.9271 - val_loss: 0.3794 - val_accuracy: 0.8804
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2090 - accuracy: 0.9275 - val_loss: 0.3814 - val_accuracy: 0.8790
Epoch 3/10
391/390 [==============================] - 86s 219ms/step - loss: 0.2078 - accuracy: 0.9266 - val_loss: 0.3826 - val_accuracy: 0.8792
Epoch 4/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2065 - accuracy: 0.9278 - val_loss: 0.3835 - val_accuracy: 0.8788
Epoch 5/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2078 - accuracy: 0.9262 - val_loss: 0.3826 - val_accuracy: 0.8790
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2086 - accuracy: 0.9270 - val_loss: 0.3830 - val_accuracy: 0.8792
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2079 - accuracy: 0.9275 - val_loss: 0.3825 - val_ac

In [12]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_220epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0000001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_230epochs.h5'))

Epoch 1/10
391/390 [==============================] - 86s 220ms/step - loss: 0.2070 - accuracy: 0.9283 - val_loss: 0.3819 - val_accuracy: 0.8791
Epoch 2/10
391/390 [==============================] - 85s 219ms/step - loss: 0.2054 - accuracy: 0.9287 - val_loss: 0.3818 - val_accuracy: 0.8792
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2074 - accuracy: 0.9272 - val_loss: 0.3823 - val_accuracy: 0.8792
Epoch 4/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2072 - accuracy: 0.9265 - val_loss: 0.3814 - val_accuracy: 0.8795
Epoch 5/10
391/390 [==============================] - 85s 219ms/step - loss: 0.2095 - accuracy: 0.9271 - val_loss: 0.3829 - val_accuracy: 0.8786
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2079 - accuracy: 0.9281 - val_loss: 0.3823 - val_accuracy: 0.8789
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2085 - accuracy: 0.9272 - val_loss: 0.3807 - val_ac

In [13]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_230epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_240epochs.h5'))

Epoch 1/10
391/390 [==============================] - 86s 220ms/step - loss: 0.2113 - accuracy: 0.9245 - val_loss: 0.3870 - val_accuracy: 0.8771
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2130 - accuracy: 0.9260 - val_loss: 0.4107 - val_accuracy: 0.8719
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2150 - accuracy: 0.9246 - val_loss: 0.3936 - val_accuracy: 0.8766
Epoch 4/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2128 - accuracy: 0.9247 - val_loss: 0.3988 - val_accuracy: 0.8765
Epoch 5/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2096 - accuracy: 0.9269 - val_loss: 0.3807 - val_accuracy: 0.8807
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2110 - accuracy: 0.9252 - val_loss: 0.3818 - val_accuracy: 0.8801
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.2118 - accuracy: 0.9261 - val_loss: 0.3922 - val_ac

In [14]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_240epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_250epochs.h5'))

Epoch 1/10
391/390 [==============================] - 86s 220ms/step - loss: 0.1612 - accuracy: 0.9447 - val_loss: 0.3827 - val_accuracy: 0.8804
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1590 - accuracy: 0.9449 - val_loss: 0.3883 - val_accuracy: 0.8795
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1569 - accuracy: 0.9460 - val_loss: 0.3925 - val_accuracy: 0.8780
Epoch 4/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1566 - accuracy: 0.9457 - val_loss: 0.3749 - val_accuracy: 0.8807
Epoch 5/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1561 - accuracy: 0.9455 - val_loss: 0.3712 - val_accuracy: 0.8816
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1573 - accuracy: 0.9449 - val_loss: 0.3715 - val_accuracy: 0.8849
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1581 - accuracy: 0.9456 - val_loss: 0.3800 - val_ac

In [15]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_260epochs.h5'))

Epoch 1/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1524 - accuracy: 0.9469 - val_loss: 0.3701 - val_accuracy: 0.8845
Epoch 2/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1543 - accuracy: 0.9462 - val_loss: 0.3691 - val_accuracy: 0.8848
Epoch 3/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1536 - accuracy: 0.9461 - val_loss: 0.3690 - val_accuracy: 0.8859
Epoch 4/10
391/390 [==============================] - 85s 217ms/step - loss: 0.1521 - accuracy: 0.9464 - val_loss: 0.3850 - val_accuracy: 0.8815
Epoch 5/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1531 - accuracy: 0.9476 - val_loss: 0.3802 - val_accuracy: 0.8843
Epoch 6/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1508 - accuracy: 0.9469 - val_loss: 0.3827 - val_accuracy: 0.8834
Epoch 7/10
391/390 [==============================] - 85s 218ms/step - loss: 0.1515 - accuracy: 0.9465 - val_loss: 0.3807 - val_ac

In [5]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_260epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_270epochs.h5'))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
391/390 [==============================] - 50s 128ms/step - loss: 0.1541 - accuracy: 0.9451 - val_loss: 0.3879 - val_accuracy: 0.8819
Epoch 2/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1509 - accuracy: 0.9480 - val_loss: 0.3815 - val_accuracy: 0.8842
Epoch 3/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1506 - accuracy: 0.9481 - val_loss: 0.3821 - val_accuracy: 0.8831
Epoch 4/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1508 - accuracy: 0.9472 - val_loss: 0.3806 - val_accuracy: 0.8836
Epoch 5/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1486 - accuracy: 0.9480 - val_loss: 0.3823 - val_accuracy: 0.8814
Epoch 6/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1492 - accuracy: 0.9473 - val_loss: 0.3940 - val_accuracy: 0.8799
Epoch 7/10
391/390 [==============================] - 

In [6]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_270epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_280epochs.h5'))

Epoch 1/10
391/390 [==============================] - 49s 124ms/step - loss: 0.1349 - accuracy: 0.9532 - val_loss: 0.3741 - val_accuracy: 0.8838
Epoch 2/10
391/390 [==============================] - 48s 123ms/step - loss: 0.1328 - accuracy: 0.9543 - val_loss: 0.3691 - val_accuracy: 0.8856
Epoch 3/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1323 - accuracy: 0.9548 - val_loss: 0.3743 - val_accuracy: 0.8845
Epoch 4/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1279 - accuracy: 0.9560 - val_loss: 0.3715 - val_accuracy: 0.8852
Epoch 5/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1316 - accuracy: 0.9532 - val_loss: 0.3816 - val_accuracy: 0.8836
Epoch 6/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1316 - accuracy: 0.9539 - val_loss: 0.3820 - val_accuracy: 0.8848
Epoch 7/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1308 - accuracy: 0.9542 - val_loss: 0.3811 - val_ac

In [7]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_280epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_290epochs.h5'))

Epoch 1/10
391/390 [==============================] - 49s 124ms/step - loss: 0.1197 - accuracy: 0.9582 - val_loss: 0.3701 - val_accuracy: 0.8882
Epoch 2/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1195 - accuracy: 0.9582 - val_loss: 0.3649 - val_accuracy: 0.8866
Epoch 3/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1196 - accuracy: 0.9577 - val_loss: 0.3754 - val_accuracy: 0.8863
Epoch 4/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1162 - accuracy: 0.9602 - val_loss: 0.3650 - val_accuracy: 0.8889
Epoch 5/10
391/390 [==============================] - 49s 124ms/step - loss: 0.1178 - accuracy: 0.9601 - val_loss: 0.3635 - val_accuracy: 0.8904
Epoch 6/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1125 - accuracy: 0.9615 - val_loss: 0.3636 - val_accuracy: 0.8883
Epoch 7/10
391/390 [==============================] - 49s 124ms/step - loss: 0.1140 - accuracy: 0.9605 - val_loss: 0.3668 - val_ac

In [11]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
path = os.path.abspath('gdrive/My Drive/CNN_CIFAR')
path = os.path.join(path, 'Cifar_training')
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.load_weights(os.path.join(path, 'model_new_290epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0001)
checkpoint = ModelCheckpoint(os.path.join(path, 'cnn_cifar_model_new1.hdf5'), monitor = 'val_acc')
datagen = ImageDataGenerator(rotation_range = 15)
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size), steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint])
model.save_weights(os.path.join(path, 'model_new_300epochs.h5'))

Epoch 1/10
391/390 [==============================] - 49s 125ms/step - loss: 0.1119 - accuracy: 0.9613 - val_loss: 0.3632 - val_accuracy: 0.8881
Epoch 2/10
391/390 [==============================] - 49s 126ms/step - loss: 0.1093 - accuracy: 0.9629 - val_loss: 0.3631 - val_accuracy: 0.8886
Epoch 3/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1092 - accuracy: 0.9618 - val_loss: 0.3632 - val_accuracy: 0.8890
Epoch 4/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1093 - accuracy: 0.9621 - val_loss: 0.3627 - val_accuracy: 0.8889
Epoch 5/10
391/390 [==============================] - 49s 124ms/step - loss: 0.1102 - accuracy: 0.9622 - val_loss: 0.3644 - val_accuracy: 0.8886
Epoch 6/10
391/390 [==============================] - 48s 124ms/step - loss: 0.1085 - accuracy: 0.9629 - val_loss: 0.3625 - val_accuracy: 0.8892
Epoch 7/10
391/390 [==============================] - 49s 125ms/step - loss: 0.1077 - accuracy: 0.9626 - val_loss: 0.3634 - val_ac

In [12]:
from tensorflow.keras.models import load_model
model = load_model('/content/gdrive/My Drive/CNN_CIFAR/Cifar_training/cnn_cifar_model_new1.hdf5')
model.evaluate(X_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.3620 - accuracy: 0.8894


[0.3620176315307617, 0.8894000053405762]